# Investigación profunda

Este es uno de los usos más típicos de Agentics en direferentes empresas.
La invetigación profunda o **Deep Research** cómo lo encontrarás más veces son las funcionalidades avanzadas que permite a los asistentes de IA realizar una investigación más exhaustiva sobre temas más complejos, analizando y sintetizando la informaicón de múltiples fuentes para generar informes detallados. Sería como tener un asistente de investigación virtual que puede leer y comprender grandes cantidades de informacio´, indentificar patrones y extraer conclusiones.

In [3]:
import os
from dotenv import load_dotenv
import asyncio
from typing import Optional, Dict, Any

import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content

from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings

from pydantic import BaseModel, Field
from IPython.display import display, Markdown


In [4]:
load_dotenv(override=True)

True

## Herramietnas alojadas en OpenAI

EL SDK de OpenAI inlcuye las siguientes herramientas alojadas

* Herramienta `WebSerachTool` perime a un agente buscar en la web.
* Herramienta `FileSearchTool`permite recuperar información de sus almacenes de vectores de OpenAI.
* La herramietna `ComputerTool`permite automatizar tareas informáticas como tomar captras y hacer clic.


Los precios de estas herramientas están en [Platform OpenAI](https://platform.openai.com/docs/pricing#web-search)

In [5]:
INSTRUCTIONS = """Eres un asistente de investigación. Dado un término de búsqueda, buscas en la web ese término y producces una descripción concisa de los resultados. La descripcíon debe tener 2-3 párrafos y menos de 300 palabras. Captura los puntos principales. Escribe de manera concisa, no es necesario tener frases completas o buena grámatica. Esto será consumido por alguien que está sintetizaddo a un informe, por lo que es vital que captures la esencia e ignores cualquier fluff. No incluyas ningún comentaio adicional más que la descripción en sí"""

search_agent = Agent(
    name="Asistente de búsqueda",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [6]:
message = "Últimos frameworks de agentes IA en 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

En 2025, emergen varios frameworks de agentes de inteligencia artificial (IA) que facilitan la creación y gestión de sistemas autónomos y colaborativos. LangChain, por ejemplo, se destaca por su capacidad para desarrollar aplicaciones basadas en modelos de lenguaje, soportando arquitecturas de agentes complejas y memoria a largo plazo. CrewAI, respaldado por Andrew Ng, permite la formación de equipos de IA donde cada agente tiene roles y objetivos específicos, colaborando para realizar tareas complejas. Por otro lado, AutoGen de Microsoft es un marco de código abierto que facilita la cooperación entre múltiples agentes de IA para resolver tareas, acelerando el desarrollo e investigación en este ámbito. ([brainandcode.tech](https://brainandcode.tech/blogs/el-blog-de-brain/7-frameworks-de-ia-open-source-que-estan-destacando-en-2025?utm_source=openai), [kwfoundation.org](https://kwfoundation.org/blog/2025/02/19/el-agente-de-inteligencia-artificial-significara-el-fin-del-saas/?utm_source=openai))

Además, se presentan iniciativas como AgentOrchestra, un framework jerárquico de múltiples agentes para la resolución de tareas generales, que integra planificación de alto nivel con colaboración modular de agentes. SwarmAgentic propone una generación totalmente automatizada de sistemas agénticos mediante inteligencia de enjambre, optimizando la funcionalidad y colaboración de los agentes de manera conjunta. AutoAgent, por su parte, es un sistema autónomo que permite la creación y despliegue de agentes de IA sin necesidad de codificación, facilitando la adopción de agentes de IA por usuarios sin experiencia técnica. ([arxiv.org](https://arxiv.org/abs/2506.12508?utm_source=openai), [arxiv.org](https://arxiv.org/abs/2506.15672?utm_source=openai), [arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

Estas herramientas reflejan la rápida evolución de la IA agéntica, ofreciendo soluciones más accesibles y eficientes para la automatización de tareas y la colaboración entre agentes inteligentes. 

## Creando salidas estructuradas

Vamos a tener una descripción más concreataas de los campos

In [7]:
HOW_MANY_SEARCHES = 3

INSCTRUCTIONS = f"""Eres un asistente de investigación. Dado un término de búsqueda, buscas en la web ese término y produces una descripción concisa de los resultados. La descripción debe tener 2-3 párrafos y menos de 300 palabras. Captura los puntos principales. Escribe de manera concisa, no es necesario tener frases completas o buena gramática. Esto será consumido por alguien que está sintetizando un informe, por lo que es vital que captures la esencia e ignores cualquier fluff. No incluyas ningún comentario adicional más que la descripción en sí. Las Salida {HOW_MANY_SEARCHES} términos para consultar"""

# Usamos Pydantic para definir el esquema que meustra en  la respuesta es lo conocido como "salida estructurada"


class WebSearchItem(BaseModel):
    reason :str = Field(description="Tu razonamietno para por qué esta búsqueda es inportante para la consulta")

    query: str = Field(description="El término de búsqueda que se usará para buscar en la web")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(
        description="Una lista de términos de búsqueda que se usarán para buscar en la web. Cada término debe tener una razón por la cual es importante para la consulta"
    )

planner_agent = Agent(
    name="Agente de planificaciones",
    instructions=INSCTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [8]:
message = "Últimos frameworks de agentes IA en 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='Identificar las tendencias actuales y futuras en el desarrollo de agentes IA.', query='frameworks de agentes IA 2025'), WebSearchItem(reason='Conocer las principales herramientas y tecnologías que marcarán el desarrollo de la IA.', query='nuevas tecnologías en IA 2025'), WebSearchItem(reason='Comprender cómo estos frameworks afectan la adopción de IA en diversas industrias.', query='impacto de frameworks de agentes IA en la industria 2025')]


In [9]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """Envía un correo electrónico con el asunto y el contenido HTML proporcionados.

    Args:
        subject (str): Asunto del correo electrónico.
        html (str): HTML del correo electrónico

    Returns:
        Dict[str, str]: Correco en html con el asunto y el contenido.
    """

    sg = sendgrid.SendGridAPIClient(os.getenv("SENDGRID_API_KEY"))
    from_email = Email("pichu_2707@hotmail.com")
    to_email = To("pichu2707@gmail.com")
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()

    response = sg.client.mail.send.post(request_body=mail)
    return {"status":"success"}


In [10]:
send_email

FunctionTool(name='send_email', description='Envía un correo electrónico con el asunto y el contenido HTML proporcionados.', params_json_schema={'properties': {'subject': {'description': 'Asunto del correo electrónico.', 'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001C53D866840>, strict_json_schema=True, is_enabled=True)

In [11]:
INSTRUCTIONS = """Eres capaz de enviar correos electr´ponico HTML bien formateado basado en un informe detallado. Se te proporcionará un informe detallado.Debes usar tu herramienta para enviar el corre electrónico, proporcionando el informe convertido en HTML limpio, bien presentado y con un asiunto adecuado. El correo electrónico debe ser claro, conciso y fácil de leer. No incluyas ningún comentario adicional más que el correo electrónico en sí."""

email_agent = Agent(
    name="Agente de correo eletrónonico",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [13]:
INSTRUCTIONS = (
    "Eres un investigador senior encargado de escribir un infroome coherente para una consulta de investigación"
    "Se te proporciona la consulta original y algunas investigaciones iniciales realizadas po run asistente de investigación"
    "Primero, debes elaborar un esquema para el informe que describa la estructura"
    "Crea un flujo del informe. Luego, genera el informe y devuelve como tu salida final.\n"
    "La saida final debe estar en formato markdown, y debe ser larga y detallada"
    "para 5-10 páginas de contenido de al menos 1000 palabras"
)

class ReportData(BaseModel):
    short_summary: str = Field(description="Un resumen de 2-3 párrafos de los resultrados")

    markdown_report: str = Field(description="El informe final en formato markdown")

    follow_up_questions: list[str] = Field(description="Temas ssugeridos para investigar más")

writer_agent = Agent(
    name="Agente de escritura",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

Creamos tres funciones para plinificar y ejecutar las búsquedas

In [14]:
async def plan_searches(query: str):
    """Utilice un agente para planificar (planner_agent) las búsquedas necesarias para responder a una consulta.

    Args:
        query (str): Query de búsqueda para la que se planificarán las búsquedas.
    """
    print("Planificando búsquedas para:", query)
    result = await Runner.run(planner_agent, query)
    print(f"Se realizarán {len(result.final_output.searches)} búsquedas")
    return result.final_output


async def perform_searches(search_plan: WebSearchPlan):
    """Llama a search() para cada elemento en el plan de búsqueda.

    Args:
        searches (list[WebSearchItem]): Lista de términos de búsqueda a realizar.

    """
    print("Buscando ...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Búsquedas completadas")
    return results

async def search(item: WebSearchItem):
    """Realiza una búsqueda en la web utilizando el término de búsqueda proporcionado.

    Args:
        item (WebSearchItem): Elemento que contiene el término de búsqueda y la razón.

    Returns:
        str: Resultado de la búsqueda.
    """
    input = f"Término de búsqueda para ejecutra la búsqueda web para cada elemento"
    result = await Runner.run(search_agent, input)
    return result.final_output

Las siguientes funciones escriben un informe y lo envían por correo electrónico.

In [17]:
async def write_reporte(query: str, search_results: list[str]):
    """Escribe un informe basado en los resultados de búsqueda.

    Args:
        query (str): Consulta original.
        search_results (list[str]): Resultados de búsqueda obtenidos.

    Returns:
        ReportData: Datos del informe generado.
    """
    print("Escribiendo informe...")
    input = f"Consulta original: {query}\nResultados de búsqueda: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Informe escrito")
    return result.final_output

async def send_email(report: ReportData):
    """Envía un correo electrónico con el informe generado.

    Args:
        report (ReportData): Datos del informe a enviar por correo electrónico.
    """
    print("Enviando correo electrónico...")
    subject = "Informe de investigación"
    html_body = f"<h1>Informe de investigación</h1><p>{report.short_summary}</p><div>{report.markdown_report}</div>"
    result = await Runner.run(email_agent, report.markdown_report)
    print("Correo electrónico enviado")
    return result.final_output

Marchando con el código

In [18]:
query = "Últimos frameworks de agentes IA en 2025"

with trace("Investicación"):
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_reporte(query, search_results)
    email_result = await send_email(report)
    print("El proceso ha finalizado con éxito!!!")

Planificando búsquedas para: Últimos frameworks de agentes IA en 2025
Se realizarán 3 búsquedas
Buscando ...
Búsquedas completadas
Escribiendo informe...
Informe escrito
Enviando correo electrónico...
Correo electrónico enviado
El proceso ha finalizado con éxito!!!
